In [29]:
# 📦 Imports
import pandas as pd
import numpy as np
from scipy.stats import zscore

# 🔧 Load the latest merged dataset
df = pd.read_csv("../Easy/layman_fred_batch.csv", index_col="date", parse_dates=True)

# Add NAIRU (natural rate of unemployment) if missing
if "layman__nairu_estimate" not in df.columns:
    try:
        nairu_series = fred.get_series("NROU", observation_start=start_date, observation_end=end_date)
        df["layman__nairu_estimate"] = nairu_series.resample(freq).ffill().reindex(df.index)
        print("✅ Added: layman__nairu_estimate (from NROU)")
    except Exception as e:
        print(f"❌ Could not load NAIRU (NROU): {e}")

# Add Core CPI from correct code
if "layman__core_cpi" not in df.columns:
    try:
        core_cpi_series = fred.get_series("CPILFESL", observation_start=start_date, observation_end=end_date)
        df["layman__core_cpi"] = core_cpi_series.resample(freq).ffill().reindex(df.index)
        print("✅ Added: layman__core_cpi (from CPILFESL)")
    except Exception as e:
        print(f"❌ Failed to load Core CPI: {e}")

# Ensure U6 underemployment rate is present
if "layman__u6_underemployment_rate" not in df.columns:
    try:
        u6 = fred.get_series("U6RATE", observation_start=start_date, observation_end=end_date)
        df["layman__u6_underemployment_rate"] = u6.resample(freq).ffill().reindex(df.index)
        print("✅ Added: layman__u6_underemployment_rate (from U6RATE)")
    except Exception as e:
        print(f"❌ Failed to load U6RATE: {e}")

# Ensure part-time for economic reasons is present
if "layman__part_time_econ_reasons" not in df.columns:
    try:
        pte = fred.get_series("LNS12032194", observation_start=start_date, observation_end=end_date)
        df["layman__part_time_econ_reasons"] = pte.resample(freq).ffill().reindex(df.index)
        print("✅ Added: layman__part_time_econ_reasons (from LNS12032194)")
    except Exception as e:
        print(f"❌ Failed to load part-time econ reasons: {e}")

# 👶 Childcare CPI Fallback Logic
if "layman__cpi_childcare" not in df.columns:
    try:
        # Preferred: Day care and nursery school
        childcare = fred.get_series("CUSR0000SEEB03", observation_start=start_date, observation_end=end_date)
        df["layman__cpi_childcare"] = childcare.resample(freq).ffill().reindex(df.index)
        print("✅ Added: layman__cpi_childcare (from CUSR0000SEEB03)")
    except Exception:
        try:
            # Fallback: Broader education & communication CPI
            fallback = fred.get_series("CUUR0000SEEB", observation_start=start_date, observation_end=end_date)
            df["layman__cpi_childcare"] = fallback.resample(freq).ffill().reindex(df.index)
            print("⚠️ Fallback used: layman__cpi_childcare (from CUUR0000SEEB)")
        except Exception as e:
            print(f"❌ Failed both childcare CPI options: {e}")

# 🧭 Yield Curve Spread (T10Y2Y)
if "layman__yield_curve_spread" not in df.columns:
    try:
        spread = fred.get_series("T10Y2Y", observation_start=start_date, observation_end=end_date)
        df["layman__yield_curve_spread"] = spread.resample(freq).ffill().reindex(df.index)
        print("✅ Added: layman__yield_curve_spread (from T10Y2Y)")
    except Exception as e:
        print(f"❌ Could not load yield curve spread: {e}")
        
# =========================
# 💡 Derived Theoreticals with Layman Names
# =========================

# 1. Unused Worker Slack (Okun’s Law Proxy)
df["layman__unused_worker_slack"] = df["layman__unemployment_rate"] - df["layman__nairu_estimate"]
# → Theory: Okun’s Law (output gap ~ difference from NAIRU)

# 2. Interest Rate Misalignment (Taylor Rule Gap Proxy)
taylor_ideal = (
    1 + 1.5 * (df["layman__core_cpi"] - 2) +
    0.5 * (df["layman__nairu_estimate"] - df["layman__unemployment_rate"])
)
df["layman__interest_policy_gap"] = taylor_ideal - df["layman__federal_funds_rate"]
# → Theory: Taylor Rule (expected interest rate minus actual Fed rate)

# 3. Labor Underutilization Score
df["layman__labor_underutilization_index"] = (
    0.4 * df["layman__u6_underemployment_rate"].rank(pct=True) +
    0.4 * df["layman__unemployment_rate"].rank(pct=True) +
    0.2 * df["layman__part_time_econ_reasons"].rank(pct=True)
)
# → Concept: Broader labor market slack

# 4. Housing Burden Score
df["layman__housing_affordability_index"] = (
    (1 / (df["layman__median_home_price"] * df["layman__mortgage_rate_30yr"])) *
    df["layman__personal_savings_rate"]
)
# → Concept: Ratio of income buffer to home cost burden

# 5. Youth Chaos Exposure Index
df["layman__youth_volatility_exposure"] = (
    0.5 * df["layman__youth_unemployment_rate"].rank(pct=True) +
    0.3 * (1 - df["layman__consumer_sentiment_index"].rank(pct=True)) +
    0.2 * df["layman__oil_price"].rank(pct=True)
)
# → Concept: Youth vulnerability to economic swings

# 6. Service Sector Cost Chaos Index
childcare_col = "layman__cpi_childcare" if "layman__cpi_childcare" in df.columns else "layman__education_and_childcare_cpi"
df["layman__service_sector_cost_chaos"] = (
    0.4 * df["layman__cpi_used_cars"].rank(pct=True) +
    0.3 * df["layman__cpi_childcare"].rank(pct=True) +  # ← use correct fallback column
    0.3 * df["layman__cpi_event_admission"].rank(pct=True)
)
# → Concept: Cost instability in essential services

# 7. Recession Warning Blend
recession_signals = df[[
    "layman__yield_curve_spread",
    "layman__unemployment_rate",
    "layman__total_consumer_credit",
    "layman__consumer_sentiment_index"
]]
df["layman__recession_signal_score"] = recession_signals.apply(zscore, nan_policy='omit').mean(axis=1)
# → Concept: Multi-signal early warning system

# 🧼 Clean edges
df = df.bfill().ffill()

# 💾 Save
df.to_csv("layman_fred_theoreticals_enriched.csv")
print("✅ Saved: layman_fred_theoreticals_enriched.csv with theoretical indices")

✅ Added: layman__nairu_estimate (from NROU)
✅ Added: layman__core_cpi (from CPILFESL)
✅ Added: layman__u6_underemployment_rate (from U6RATE)
✅ Added: layman__part_time_econ_reasons (from LNS12032194)
⚠️ Fallback used: layman__cpi_childcare (from CUUR0000SEEB)
✅ Added: layman__yield_curve_spread (from T10Y2Y)
✅ Saved: layman_fred_theoreticals_enriched.csv with theoretical indices
